In [1]:
!pip install "transformers >= 4.26.0"

In [2]:
import matplotlib.pyplot as plt
import json
from PIL import Image
from transformers import BlipProcessor, BlipForQuestionAnswering
import torch
from tqdm import tqdm
import torch
from torch.utils.data import Dataset
import warnings
warnings.filterwarnings("ignore")

In [3]:
file = open("/kaggle/input/daquar/DAQUAR/question_answer.txt", "r")
question_answer = file.readlines()

In [4]:
data = []
for i in range(0, len(question_answer), 2):
    question_break = question_answer[i].strip().split(" ")
    question = ' '.join(question_break[:-4] + [question_break[-1]])
    answer = question_answer[i+1].strip()
    image_dir = f"/kaggle/input/daquar/DAQUAR/images/{question_break[-2]}.png"
    data.append({
        'question': question,
        'answer': answer,
        'image_dir': image_dir
    })

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
class DAQUAR(Dataset):
    def __init__(self, data, processor):
        self.data = data
        self.processor = processor
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        question, image_dir = self.data[idx]['question'], self.data[idx]['image_dir']
        raw_image = Image.open(image_dir).convert('RGB')   
        inputs = self.processor(raw_image, question, return_tensors="pt")
        return inputs
    def add_prediction(self, idx, out):
        prediction = self.processor.decode(out[0], skip_special_tokens=True)
        self.data[idx]['prediction'] = prediction

In [7]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to(device)

2024-05-12 05:14:17.839028: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-12 05:14:17.839126: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-12 05:14:17.992946: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

In [8]:
daquar = DAQUAR(data, processor)

In [9]:
for i in tqdm(range(len(daquar))):
    inputs = daquar[i].to(device)
    out = model.generate(**inputs)
    daquar.add_prediction(i, out)

100%|██████████| 12468/12468 [22:10<00:00,  9.37it/s]


In [11]:
json.dump(daquar.data, open(f'/kaggle/working/Prediction.json', 'w'), indent=4)